# IBM Auto AI RAG 사용하기
IBM Auto AI RAG를 사용하여 RAG를 만들고 배포한 AI 서비스를 LangChain과 결합하여 최종 응답을 처리합니다.

# 라이브러리 설치하기
Lab에서 Auto AI RAG를 사용하기 위해서는 아래의 라이브러리를 설치해야 합니다.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!pip install langchain==0.3.0
!pip install langchain --upgrade

!pip install langchain_openai
!pip install langchain_community

!pip install -U langchain-ibm

라이브러리 설치가 완료 되었으면 노트북 커널을 Restart 합니다.
메뉴에서 `kernel > restart` 를 실행합니다.

# IBM Auto RAG AI 서비스를 적용한 RAG 시스템 구현


In [ ]:
# LangChain Document 객체로 변환하는 함수
def convert_to_documents(reference_docs):
    documents = []
    for doc in reference_docs:
        page_content = doc.get("page_content", "")
        metadata = doc.get("metadata", {})
        
        # LangChain Document 객체 생성
        document = Document(
            page_content=page_content,
            metadata=metadata
        )
        documents.append(document)
    return documents

"CLOUD_API_KEY"는 IBM Cloud의 watsonx 서비스에서 API KEY 정보를 가져옵니다.   
"MY_PROJECT_ID"는 IBM Cloud의 watsonx 서비스에서 프로젝트 ID 정보를 가져옵니다.


In [ ]:
# IBM Cloud API Key와 URL 설정
api_key = "<CLOUD_API_KEY>"
# region에 따라 주소가 다를 수 있습니다. 주소를 확인해 주세요.
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com" 
project_id = "<MY_PROJECT_ID>"

# 배포한 AutoAI RAG 모델의 URL
auto_ai_rag_url = "<IBM_AUTOAI_RAG_URL>"
auto_ai_rag_stream_url = "<IBM_AUTOAI_RAG_STREAM_URL>"

In [ ]:
user_input = '기업의 생성형AI 도입 현황은?'
# user_input = "기업에서 생성형 AI를 도입하는 가치는?"
# user_input = "기업에서 생성형 AI를 도입하기 위해 고려사항은?"
# user_input = "기업에서 생성형 AI 도입을 가로막는 가장 큰 장애물은?"

In [ ]:
import requests
import json

API_KEY = api_key
token_response = requests.post(
    'https://iam.cloud.ibm.com/identity/token',
    data={
        "apikey": API_KEY,
        "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
    }
)
mltoken = token_response.json().get("access_token")
if not mltoken:
    raise Exception("토큰 발급 실패!")

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {mltoken}',
    'Accept': 'text/event-stream'  # 스트리밍 응답 처리용 헤더
}

payload_scoring = {
    "messages": [
        {
            "role": "user",
            "content": user_input
        }
    ],
    "parameters": {
        "max_tokens": 1024,
        "temperature": 0.7
    }
}

response = requests.post(
    auto_ai_rag_stream_url,
    headers=headers,
    json=payload_scoring,
    stream=True
)

print("\n  AI 응답:")
full_text = ""
reference_docs = []

try:
    for line in response.iter_lines(decode_unicode=True):
        if line.startswith("data: "):
            json_str = line[6:]
            if json_str.strip() == "[DONE]":
                break
            try:
                data = json.loads(json_str)

                # content 추가
                delta = data.get("choices", [{}])[0].get("delta", {})
                content_piece = delta.get("content", "")
                if content_piece:
                    print(content_piece, end="", flush=True)
                    full_text += content_piece

                # reference_documents 저장
                if "reference_documents" in data.get("choices", [{}])[0]:
                    reference_docs = data["choices"][0]["reference_documents"]

            except json.JSONDecodeError:
                print(f"\n JSON 파싱 오류: {json_str}")
except Exception as e:
    print(f"\n 스트리밍 중 예외 발생: {e}")

print("\n  인용된 문서 내용:")
for idx, doc in enumerate(reference_docs):
    try:
        page_content = doc.get("page_content", "")
        print(f"\n[문서 {idx + 1}]\n{page_content}")
    except Exception as e:
        print(f"문서 {idx + 1} 출력 실패: {e}")



# Rangchain 을 사용한 RAG 시스템 구현
Langchain은 다양한 데이터 소스와 AI 모델을 연결하여 RAG 시스템을 구축하는 데 유용한 라이브러리입니다. Langchain을 사용하면 데이터 소스에서 정보를 검색하고, AI 모델을 사용하여 새로운 텍스트를 생성하는 작업을 쉽게 수행할 수 있습니다.

In [ ]:
import requests
import json

def call_ai_service (user_input):

    API_KEY = api_key
    token_response = requests.post(
        'https://iam.cloud.ibm.com/identity/token',
        data={
            "apikey": API_KEY,
            "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
        }
    )
    mltoken = token_response.json().get("access_token")
    if not mltoken:
        raise Exception("토큰 발급 실패!")

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {mltoken}',
        'Accept': 'text/event-stream'  # 스트리밍 응답 처리용 헤더
    }

    payload_scoring = {
        "messages": [
            {
                "role": "user",
                "content": user_input
            }
        ],
        "parameters": {
            "max_tokens": 1024,
            "temperature": 0.7
        }
    }

    response = requests.post(
        auto_ai_rag_stream_url,
        headers=headers,
        json=payload_scoring,
        stream=True
    )

    # print("\n  AI 응답:")
    full_text = ""
    reference_docs = []

    try:
        for line in response.iter_lines(decode_unicode=True):
            if line.startswith("data: "):
                json_str = line[6:]
                if json_str.strip() == "[DONE]":
                    break
                try:
                    data = json.loads(json_str)

                    # content 추가
                    delta = data.get("choices", [{}])[0].get("delta", {})
                    content_piece = delta.get("content", "")
                    if content_piece:
                        # print(content_piece, end="", flush=True)
                        full_text += content_piece

                    # reference_documents 저장
                    if "reference_documents" in data.get("choices", [{}])[0]:
                        reference_docs = data["choices"][0]["reference_documents"]

                except json.JSONDecodeError:
                    print(f"\n JSON 파싱 오류: {json_str}")
    except Exception as e:
        print(f"\n 스트리밍 중 예외 발생: {e}")

    # print("\n  인용된 문서 내용:")
    for idx, doc in enumerate(reference_docs):
        try:
            page_content = doc.get("page_content", "")
            # print(f"\n[문서 {idx + 1}]\n{page_content}")
        except Exception as e:
            print(f"문서 {idx + 1} 출력 실패: {e}")
    return reference_docs


watsionx.ai는 IBM의 AI 플랫폼으로, 다양한 AI 모델과 서비스를 제공합니다. watsonx.ai의 AI 모델에 연결하고, 데이터를 검색하고, 새로운 텍스트를 생성하는 작업을 수행할 수 있습니다.

In [ ]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_ibm import WatsonxLLM


import os
os.environ["WATSONX_URL"] = ibm_cloud_url
os.environ["WATSONX_APIKEY"] = api_key
# os.environ["WATSONX_TOKEN"] = ""

params = {
    "decoding_method": "greedy",
    "temperature": 0.7,
    "min_new_tokens": 50,
    "max_new_tokens": 1000
}

# WatsonxLLM 모델 초기화
model_llm = WatsonxLLM(
    model_id="meta-llama/llama-3-3-70b-instruct",
    project_id=project_id,
    params=params
)

배포한 Auto RAG AI 서비스에서 데이터를 검색 후 Langchain과 연동하기 위해 데이타 타입을 Document 형식으로 변환합니다.

In [ ]:
user_input = '기업의 생성형AI 도입 현황은?'
# user_input = "기업에서 생성형 AI를 도입하는 가치는?"
# user_input = "기업에서 생성형 AI를 도입하기 위해 고려사항은?"
# user_input = "기업에서 생성형 AI 도입을 가로막는 가장 큰 장애물은?"


reference_docs = call_ai_service(user_input)
# 변환된 문서 리스트
docs_search = convert_to_documents(reference_docs)

watsonx에 배포한 AI 서비스의 응답 결과를 Langchain과 결합하여 RAG 시스템을 구축할 수 있습니다. Langchain은 다양한 데이터 소스와 AI 모델을 연결하여 RAG 시스템을 구축하는 데 유용한 라이브러리입니다. Langchain을 사용하면 데이터 소스에서 정보를 검색하고, AI 모델을 사용하여 새로운 텍스트를 생성하는 작업을 쉽게 수행할 수 있습니다.

In [ ]:
# 프롬프트 정의
prompt = PromptTemplate.from_template("""
아래 문서를 참고해서 사용자 질문에 정확하게 한국어로 답변해줘.

문서:
{context}

질문: {input}

답변:
""")

# 문서 체인 구성
stuff_chain = create_stuff_documents_chain(model_llm, prompt)


# 실행
result = stuff_chain.invoke({"context": docs_search, "input":user_input})
print(result)